# Hands On

## Prepare

```SQL
DROP TABLE IF EXISTS tbl;
SELECT generate_series AS id, 0 AS data
INTO tbl
FROM generate_series(1,10);

/*** (detailed in VACUUM module) ***/
-- Turn autovacuum off
ALTER TABLE tbl SET (autovacuum_enabled = false);

-- Look at the page layout
DROP VIEW IF EXISTS vw_inspect_tbl_first_page;
CREATE VIEW vw_inspect_tbl_first_page AS
SELECT
	lp AS line_pointer
	,lp_off AS liner_pointer_offset
	,lp_len AS liner_pointer_length
	,lp_off - lp_len AS next_liner_pointer_offset
	,t_xmin
	,t_xmax
	,t_ctid
FROM 
	heap_page_items(get_raw_page('tbl', 0)) hp;

-- Look at the page headers
DROP VIEW IF EXISTS vw_tbl_first_page_free_space;
CREATE VIEW vw_tbl_first_page_free_space AS
SELECT 
    lower AS line_pointers_offset
    ,upper AS data_offset
    ,upper - lower AS free_space
	,pagesize AS page_size
FROM page_header(get_raw_page('tbl', 0));
```

## Fill Up A Page

```SQL
-- Check how much space a complete tuple is taking
SELECT free_space -- 7808
FROM vw_tbl_first_page_free_space;

INSERT INTO tbl VALUES (11, 1);
SELECT free_space -- 7772
FROM vw_tbl_first_page_free_space;

SELECT 7808 - 7772; -- 36 (4 for line_pointer, 32 for tuple)
```

```SQL
-- Check how much space is enough to trigger in page pruning
SELECT page_size / 10 -- 819
FROM vw_tbl_first_page_free_space;
```

```SQL
DO $$
DECLARE
	const_new_tuple_space INTEGER := 36;
	const_space_trigger INTEGER := 819;
	var_counter INTEGER := 1;
	var_current_free_space INTEGER;
BEGIN
	RAISE NOTICE 'Started';

	SELECT free_space
	INTO var_current_free_space
	FROM vw_tbl_first_page_free_space;

	WHILE var_current_free_space > const_space_trigger LOOP 
		RAISE NOTICE 'Inserting %th tuple', var_counter;
		var_counter := var_counter + 1;
		INSERT INTO tbl VALUES (100 + var_counter,0);
		SELECT free_space
		INTO var_current_free_space
		FROM vw_tbl_first_page_free_space;
	END LOOP;
END;
$$;
```

```SQL
-- Check out the current layout
SELECT * FROM vw_inspect_tbl_first_page;
SELECT * FROM vw_tbl_first_page_free_space;
```

## Update

```SQL
-- Update a same row 3 times and watch what happens
UPDATE tbl SET data = 1 WHERE id = 1;
SELECT * FROM vw_inspect_tbl_first_page;
SELECT * FROM vw_tbl_first_page_free_space;
